# Day 09. Exercise 04
# Pipelines and OOP

## 0. Imports

In [22]:
import pandas as pd
import numpy as np
import joblib
from tqdm.notebook import tqdm
import os

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

## 1. Preprocessing pipeline

Create three custom transformers, the first two out of which will be used within a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

1. `FeatureExtractor()` class:
 - Takes a dataframe with `uid`, `labname`, `numTrials`, `timestamp` from the file [`checker_submits.csv`](https://drive.google.com/file/d/14voc4fNJZiLEFaZyd8nEG-lQt5JjatYw/view?usp=sharing).
 - Extracts `hour` from `timestamp`.
 - Extracts `weekday` from `timestamp` (numbers).
 - Drops the `timestamp` column.
 - Returns the new dataframe.


2. `MyOneHotEncoder()` class:
 - Takes the dataframe from the result of the previous transformation and the name of the target column.
 - Identifies all the categorical features and transforms them with `OneHotEncoder()`. If the target column is categorical too, then the transformation should not apply to it.
 - Drops the initial categorical features.
 - Returns the dataframe with the features and the series with the target column.


3. `TrainValidationTest()` class:
 - Takes `X` and `y`.
 - Returns `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` (`test_size=0.2`, `random_state=21`, `stratified`).


In [23]:
class FeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X['timestamp'] = pd.to_datetime(X['timestamp'], errors='coerce')
        X['hour'] = X['timestamp'].dt.hour
        X['dayofweek'] = X['timestamp'].dt.weekday
        X = X.drop(columns=['timestamp'])
        return X

In [24]:
class MyOneHotEncoder(BaseEstimator, TransformerMixin):

    def __init__(self, target):
        self.target = target
        self.encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        self.categorical_features = []

    def fit(self, X, y=None):
        features = X.drop(columns=[self.target])
        self.categorical_features = features.select_dtypes(include=['object']).columns.tolist()
        if self.categorical_features:
            self.encoder.fit(features[self.categorical_features])
        return self
    
    def transform(self, X):
        X = X.copy()
        y = X[self.target]
        features = X.drop(columns=[self.target])
        if self.categorical_features:
            encoded = self.encoder.transform(features[self.categorical_features])
            encoded_cols = self.encoder.get_feature_names_out(self.categorical_features)
            encoded_df = pd.DataFrame(encoded, columns=encoded_cols, index=features.index)
            features = features.drop(columns=self.categorical_features)
            features = pd.concat([features, encoded_df], axis=1)
            df = pd.concat([features, y], axis=1)
        return df

In [25]:
class TrainValidationTest:
    def __init__(self, test_size=0.2, random_state=21, stratified=True):
        self.test_size = test_size
        self.random_state = random_state
        self.stratified = stratified
    
    def split_train_valid_test(self, X, y):
        stratify = y if self.stratified else None
        X_train_valid, X_test, y_train_valid, y_test = train_test_split(
            X, y, 
            test_size=self.test_size, 
            random_state=self.random_state,
            stratify=stratify
        )
        
        valid_size = self.test_size / (1 - self.test_size)
        stratify = y_train_valid if self.stratified else None
        X_train, X_valid, y_train, y_valid = train_test_split(
            X_train_valid, y_train_valid, 
            test_size=valid_size, 
            random_state=self.random_state,
            stratify=stratify
        )
        return X_train, X_valid, X_test, y_train, y_valid, y_test

## 2. Model selection pipeline

`ModelSelection()` class

 - Takes a list of `GridSearchCV` instances and a dict where the keys are the indexes from that list and the values are the names of the models, the example is below in the reverse order (from high-level to low-level perspective):

```
ModelSelection(grids, grid_dict)

grids = [gs_svm, gs_tree, gs_rf]

gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=jobs), where jobs you can specify by yourself

svm_params = [{'kernel':('linear', 'rbf', 'sigmoid'), 'C':[0.01, 0.1, 1, 1.5, 5, 10], 'gamma': ['scale', 'auto'], 'class_weight':('balanced', None), 'random_state':[21], 'probability':[True]}]
```

 - Method `choose()` takes `X_train`, `y_train`, `X_valid`, `y_valid` and returns the name of the best classifier among all the models on the validation set
 - Method `best_results()` returns a dataframe with the columns `model`, `params`, `valid_score` where the rows are the best models within each class of models.

```
model	params	valid_score
0	SVM	{'C': 10, 'class_weight': None, 'gamma': 'auto...	0.772727
1	Decision Tree	{'class_weight': 'balanced', 'criterion': 'gin...	0.801484
2	Random Forest	{'class_weight': None, 'criterion': 'entropy',...	0.855288
```

 - When you iterate through the parameters of a model class, print the name of that class and show the progress using `tqdm.notebook`, in the end of the cycle print the best model of that class.

```
Estimator: SVM
100%
125/125 [01:32<00:00, 1.36it/s]
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878 

Estimator: Decision Tree
100%
57/57 [01:07<00:00, 1.22it/s]
Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.801
Validation set accuracy score for best params: 0.867 

Estimator: Random Forest
100%
284/284 [06:47<00:00, 1.13s/it]
Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.855
Validation set accuracy score for best params: 0.907 

Classifier with best validation set accuracy: Random Forest
```

In [26]:
class ModelSelection:

    def __init__(self, grids, grid_dict):
        self.grids = grids
        self.grids_dict = grid_dict
        self.results = []

    def choose(self, X_train, y_train, X_valid, y_valid):
        for grid, estimator_name in zip(self.grids, self.grids_dict.values()):
            print(f'Estimator: {estimator_name}')
            
            grid.fit(X_train, y_train)
            valid_score = grid.score(X_valid, y_valid)

            self.results.append({
                'model': estimator_name,
                'params': grid.best_params_,
                'train_score': grid.best_score_,
                'valid_score': valid_score
            })
            
            print(f'Best params: {grid.best_params_}')
            print(f'Best training accuracy: {grid.best_score_:.3f}')
            print(f'Validation set accuracy score for best params: {valid_score:.3f}')
            print()

    def best_results(self):
        results_df = pd.DataFrame(self.results)
        
        best_model = results_df.loc[results_df['valid_score'].idxmax()]
        print(f'Classifier with best validation set accuracy: {best_model["model"]}')
        
        return results_df

In [27]:
svm = SVC()
tree = DecisionTreeClassifier()
rf = RandomForestClassifier()

svm_params = {
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'C': [0.01, 0.1, 1, 1.5, 5, 10],
    'gamma': ['scale', 'auto'],
    'class_weight': ['balanced', None],
    'random_state': [21],
    'probability': [True]
}

tree_params = {
    'max_depth': np.arange(1, 50),
    'class_weight': [None, 'balanced'],
    'criterion': ['gini', 'entropy'],
    'random_state': [21]
}

rf_params =  {
    'n_estimators': [5, 10, 50, 100],
    'max_depth': np.arange(1, 50),
    'class_weight': [None, 'balanced'],
    'criterion': ['gini', 'entropy'],
    'random_state': [21]
}

gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=5, n_jobs=-1)
gs_tree = GridSearchCV(estimator=tree, param_grid=tree_params, scoring='accuracy', cv=5, n_jobs=-1)
gs_rf = GridSearchCV(estimator=rf, param_grid=rf_params, scoring='accuracy', cv=5, n_jobs=-1)

grids = [gs_svm, gs_tree, gs_rf]
grids_dict = {0: 'SVC', 1: 'DecisionTreeClassifier', 2: 'RandomForestClassifier'}

## 3. Finalization

`Finalize()` class
 - Takes an estimator.
 - Method `final_score()` takes `X_train`, `y_train`, `X_test`, `y_test` and returns the accuracy of the model as in the example below:
```
final.final_score(X_train, y_train, X_test, y_test)
Accuracy of the final model is 0.908284023668639
```
 - Method `save_model()` takes a path, saves the model to this path and prints that the model was successfully saved.

In [28]:
class Finalize:
    def __init__(self, model):
        self.model = model
    
    def final_score(self, X_train, X_test, y_train, y_test):
        self.model.fit(X_train, y_train)
        score = self.model.score(X_test, y_test)
        print(f'Accuracy of the final model is: {score}')
        return score
    
    def save_model(self, path):
        joblib.dump(self.model, path)
        if os.path.exists and os.path.getsize(path) > 0:
            print('The model was successfully saved')
        else:
            print('Error: not model')


## 4. Main program

1. Load the data from the file (****name of file****).
2. Create the preprocessing pipeline that consists of two custom transformers: `FeatureExtractor()` and `MyOneHotEncoder()`:
```
preprocessing = Pipeline([('feature_extractor', FeatureExtractor()), ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
```
3. Use that pipeline and its method `fit_transform()` on the initial dataset.
```
data = preprocessing.fit_transform(df)
```
4. Get `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` using `TrainValidationTest()` and the result of the pipeline.
5. Create an instance of `ModelSelection()`, use the method `choose()` applying it to the models that you want and parameters that you want, get the dataframe of the best results.
6. create an instance of `Finalize()` with your best model, use method `final_score()` and save the model in the format: `name_of_the_model_{accuracy on test dataset}.sav`.

That is it, congrats!

In [29]:
df = pd.read_csv('../data/checker_submits.csv', parse_dates=['timestamp'])

preprocessing = Pipeline([('feature_extractor', FeatureExtractor()), ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
data = preprocessing.fit_transform(df)
data.head()

,numTrials,hour,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,uid_user_15,...,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1,dayofweek
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4


In [30]:
X = data.drop('dayofweek', axis=1)
y = data['dayofweek']
splitter = TrainValidationTest()
X_train, X_valid, X_test, y_train, y_valid, y_test = splitter.split_train_valid_test(X, y)

In [31]:
selection = ModelSelection(grids, grids_dict)
selection.choose(X_train, y_train, X_valid, y_valid)
model_df = selection.best_results()

Estimator: SVC


Best params: {'C': 10, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.834
Validation set accuracy score for best params: 0.831

Estimator: DecisionTreeClassifier
Best params: {'class_weight': None, 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.858
Validation set accuracy score for best params: 0.864

Estimator: RandomForestClassifier
Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 27, 'n_estimators': 100, 'random_state': 21}
Best training accuracy: 0.892
Validation set accuracy score for best params: 0.884

Classifier with best validation set accuracy: RandomForestClassifier


In [32]:
final = Finalize(RandomForestClassifier(class_weight=None, criterion='entropy', max_depth=27, n_estimators=100, random_state=21))
accuracy = final.final_score(X_train, X_test, y_train, y_test)
final.save_model(f'../data/final_model_{accuracy:.5f}.pkl')

Accuracy of the final model is: 0.9053254437869822
The model was successfully saved
